## Tools

In [3]:
import pandas as pd

In [4]:
from datetime import datetime
from typing import Literal

def natural_date_to_ms(date_str:str)->int:
    
    #fmt = "%Y-%m-%d %H:%M"
    fmt = "%Y-%m-%d"

    # Conversion en datetime
    dt = datetime.strptime(date_str, fmt)

    # Conversion en millisecondes
    return int(dt.timestamp() * 1000)

   
def annualized_8h(rate_8h:float, type: Literal["linear", "compound"] = "linear"):
    # Return directly the annualised rate as percentage
    if type =="compound":
        return ((1 + rate_8h/100) ** 1095 - 1)*100 
    return rate_8h * 1095 *100 #24 / 8 * 365


## Binance

### Historic Binance Rate

In [5]:
from binance.cm_futures import CMFutures
import logging
from binance.lib.utils import config_logging


def get_binance_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    
    if start_ms:
        start_ms = natural_date_to_ms(start_ms)
    if end_ms:
        end_ms = natural_date_to_ms(end_ms)
        assert(start_ms < end_ms),"End date must greater than Start Date"


    config_logging(logging, logging.DEBUG)

    cm_futures_client = CMFutures()
    temp=cm_futures_client.funding_rate(coin, **{"limit": 1000,"startTime": start_ms, "endTime": end_ms})
    #temp=cm_futures_client.funding_rate("BTCUSD_PERP", **{"limit": 1000})

    df = pd.DataFrame(temp)
    del(temp)
    # Convert types
    df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
    df['fundingRate'] = pd.to_numeric(df['fundingRate'])
    df['markPrice'] = pd.to_numeric(df['markPrice'])
    df['annualized_funding']=df['fundingRate'].apply(annualized_8h)

    df = (
    df
    .drop(columns=["symbol","fundingRate"])
    .drop_duplicates()
    .set_index('fundingTime')
    .rename(columns={'annualized_funding': 'annualized_funding_binance'})
    )
    df.index = df.index.floor('min')  # 'T' is shorthand for 'min'

    return df

## Bybit

### Historic Bybit Rate

In [8]:
import pandas as pd
from pybit.unified_trading import HTTP
import time

def get_bybit_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    
    if end_ms:
        end_ms = natural_date_to_ms(end_ms)
    if start_ms:
        start_ms = natural_date_to_ms(start_ms)
        assert(start_ms < end_ms),"End date must greater than Start Date"
        # faire le test pour verifier que l'on ne prends pas plus de 200 valeurs dans l'interval

    
    session = HTTP()
    limit = 100
    all_data = []

    current_start = start_ms
    while True:
        temp = session.get_funding_rate_history(
            category="linear",
            symbol=coin,
            startTime=current_start,
            endTime=current_start + 5760000000,
            limit=limit
        )

        result = temp["result"]["list"]
        if not result:
            break
        
        all_data.extend(result)

        # Stop if less than limit entries returned (means last page)
        if len(result) < limit:
            break

        # Move to next time window: set current_start to 1 ms after last returned timestamp
        last_timestamp = int(result[-1]['fundingRateTimestamp'])
        current_start = last_timestamp + 1

        time.sleep(0.2)  # Respect API rate limit

    df = pd.DataFrame(all_data)
    if df.empty:
        return df

    df['fundingRateTimestamp'] = pd.to_numeric(df['fundingRateTimestamp'])
    df['fundingRateTimestamp'] = pd.to_datetime(df['fundingRateTimestamp'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['annualized_funding'] = df['fundingRate'].apply(annualized_8h)

    df = (
    df
    .drop_duplicates()
    .set_index('fundingRateTimestamp')
    .rename(columns={'annualized_funding': 'annualized_funding_bybit'})
    )
    df.index = df.index.floor('min')  # type: ignore # 'T' is shorthand for 'min'

    return df


In [ ]:
from pybit.unified_trading import HTTP

def get_dft_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    # error if only start time

    if end_ms:
        end_ms = natural_date_to_ms(end_ms)
        if start_ms:
            start_ms = natural_date_to_ms(start_ms)
            assert(start_ms < end_ms),"End date must greater than Start Date"
            # faire le test pour verifier que l'on ne prends pas plus de 200 valeurs dans l'interval

    session = HTTP()
    temp=session.get_funding_rate_history(
        category="linear",
        symbol=coin,
        startTime=start_ms,
        endTime= end_ms,
    #limit integer	Limit for data size per page. [1, 200]. Default: 200
    )
    
    df = pd.DataFrame(temp["result"]['list'])
    del temp
    df['fundingRateTimestamp'] = pd.to_numeric(df['fundingRateTimestamp'])
    df['fundingRateTimestamp'] = pd.to_datetime(df['fundingRateTimestamp'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['annualized_funding']=df['fundingRate'].apply(annualized_8h)

    df = (
    df
    .drop_duplicates()
    .set_index('fundingRateTimestamp')
    .rename(columns={'annualized_funding': 'annualized_funding_dft'})
    )
    df.index = df.index.floor('min')  # 'T' is shorthand for 'min'

    return df

## OKX

### Historic OKX Rate

In [74]:
import time
import okx.PublicData as PublicData

def get_okx_funding_history(coin: str, start_ms: str = None, end_ms: str = None) -> pd.DataFrame:
    
    
    if start_ms != None and end_ms != None:
        start_ms = natural_date_to_ms(start_ms)
        end_ms = natural_date_to_ms(end_ms)

    if start_ms != None and end_ms == None:
        start_ms = natural_date_to_ms(start_ms)
        end_ms = start_ms + 7 * 24 * 3600 * 1000

    if start_ms == None and end_ms != None:
        
        end_ms = natural_date_to_ms(end_ms)
        start_ms = end_ms - 7 * 24 * 3600 * 1000
    else:
        #past 7 days
        end_ms = int(time.time() * 1000)
        start_ms = end_ms - 7 * 24 * 3600 * 1000
        

    flag = "0"  # Production trading: 0, Demo trading: 1

    publicDataAPI = PublicData.PublicAPI(flag=flag)

    # Retrieve funding rate history
    temp = publicDataAPI.funding_rate_history(
        instId=coin,
        before=start_ms,
        after=end_ms,
        #limit	String	No	Number of results per request. The maximum is 100; The default is 100
    )
    df = pd.DataFrame(temp['data'])
    del temp
    
    df['fundingTime'] = pd.to_numeric(df['fundingTime'])
    df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['annualized_funding']=df['fundingRate'].apply(annualized_8h)

    return df


In [73]:
get_okx_funding_history("ETH-USD-SWAP")
#get_okx_funding_history("BTC-USD-SWAP",start_ms = '2025-06-21') 
#get_okx_funding_history("BTC-USD-SWAP",end_ms='2025-06-27')
#get_okx_funding_history("BTC-USD-SWAP",end_ms='2025-06-27',start_ms = '2025-06-21')

DEBUG:httpcore.connection:connect_tcp.started host='www.okx.com' port=443 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001AB4D444AA0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000001AB664CA330> server_hostname='www.okx.com' timeout=5.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x000001AB66502030>
DEBUG:httpcore.http2:send_connection_init.started request=<Request [b'GET']>
DEBUG:httpcore.http2:send_connection_init.complete
DEBUG:httpcore.http2:send_request_headers.started request=<Request [b'GET']> stream_id=1
DEBUG:hpack.hpack:Adding (b':method', b'GET') to the header table, sensitive:False, huffman:True
DEBUG:hpack.hpack:Encoding 2 with 7 bits
DEBUG:hpack.hpack:Adding (b':authority', b'www.okx.com') to the header table, sensitive:False, huffman:True
DEBUG:hpack.hpack:

,formulaType,fundingRate,fundingTime,instId,instType,method,realizedRate
0,withRate,0.000031,2025-06-28 16:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000313541647633
1,withRate,0.000014,2025-06-28 08:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000135231409159
2,withRate,-0.000029,2025-06-28 00:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000287968473739
3,withRate,0.000056,2025-06-27 16:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000560573162987
4,withRate,0.000056,2025-06-27 08:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000559383318449
5,withRate,-0.000037,2025-06-27 00:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000366319000835
6,withRate,0.000070,2025-06-26 16:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000701912910958
7,withRate,0.000057,2025-06-26 08:00:00,ETH-USD-SWAP,SWAP,current_period,0.0000567331198962
8,withRate,-0.000064,2025-06-26 00:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000640593288092
9,withRate,-0.000016,2025-06-25 16:00:00,ETH-USD-SWAP,SWAP,current_period,-0.0000159787670455


### Current OKX Rate

In [ ]:
import okx.PublicData as PublicData

flag = "0"  # Production trading: 0, Demo trading: 1

publicDataAPI = PublicData.PublicAPI(flag=flag)

# Retrieve funding rate
result = publicDataAPI.get_funding_rate(
    instId="BTC-USD-SWAP",
)

Okx_Funding = pd.DataFrame(result['data'])
Okx_Funding['timestamp'] = pd.to_datetime(Okx_Funding['ts'], unit='ms')
Okx_Funding['fundingRate'] = Okx_Funding['fundingRate'].astype(float)

C:\Users\pmacr\AppData\Local\Temp\ipykernel_23352\723554402.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  Okx_Funding['timestamp'] = pd.to_datetime(Okx_Funding['ts'], unit='ms')


## Hyperliquid

### Hyperliquid history

In [2]:
import requests
import pandas as pd
import time

def get_funding_history(coin: str, start_ms: int, end_ms: int = None):
    payload = {
        "type": "fundingHistory",
        "coin": coin.upper(),
        "startTime": start_ms,
    }
    if end_ms:
        payload["endTime"] = end_ms

    resp = requests.post("https://api.hyperliquid.xyz/info", json=payload)
    resp.raise_for_status()
    data = resp.json()

    df = pd.DataFrame(data)
    if df.empty:
        return df

    df['timestamp'] = pd.to_datetime(df['time'], unit='ms')
    df['fundingRate'] = df['fundingRate'].astype(float)
    df['premium'] = df['premium'].astype(float)
    return df[['timestamp', 'fundingRate', 'premium']]




## Analysis

In [9]:
binance=get_binance_funding_history('BTCUSD_PERP')
bybi=get_bybit_funding_history("BTCPERP",start_ms="2024-08-01",end_ms="2025-06-30")
merged=binance.join(bybi,how='outer').sort_index()
merged=merged[merged.index> "2024-09-03 00:00:00"]
merged['bin_minus_bybit']=merged['annualized_funding_binance']-merged['annualized_funding_bybit']

DEBUG:root:url: https://dapi.binance.com/dapi/v1/fundingRate
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dapi.binance.com:443
DEBUG:urllib3.connectionpool:https://dapi.binance.com:443 "GET /dapi/v1/fundingRate?symbol=BTCUSD_PERP&limit=1000 HTTP/1.1" 200 None
DEBUG:root:raw response from server:[{"symbol":"BTCUSD_PERP","fundingTime":1722499200001,"fundingRate":"0.00010000","markPrice":"64276.12841843"},{"symbol":"BTCUSD_PERP","fundingTime":1722528000000,"fundingRate":"0.00010000","markPrice":"62828.70000000"},{"symbol":"BTCUSD_PERP","fundingTime":1722556800000,"fundingRate":"0.00010000","markPrice":"65293.05019887"},{"symbol":"BTCUSD_PERP","fundingTime":1722585600000,"fundingRate":"0.00010000","markPrice":"64086.62333147"},{"symbol":"BTCUSD_PERP","fundingTime":1722614400000,"fundingRate":"0.00010000","markPrice":"63339.60000000"},{"symbol":"BTCUSD_PERP","fundingTime":1722643200001,"fundingRate":"0.00010000","markPrice":"61387.80000000"},{"symbol":"BTCUSD_PERP","fundi

In [10]:
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser'  # Open in browser for full interactivity

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=merged.index,
    y=merged['bin_minus_bybit'],
    mode='lines',
    name='Binance - Bybit',
    line=dict(color='royalblue')
))

fig.add_hline(y=0, line_dash="dot", line_color="gray")

fig.update_layout(
    title='Funding Rate Difference: Binance - Bybit',
    xaxis_title='Date',
    yaxis_title='Annualized Funding Rate Difference',
    template='plotly_white',
    hovermode='x unified',
    dragmode='zoom',  # Allows zooming both horizontally and vertically
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date",
        fixedrange=False  # allow x-axis zoom
    ),
    yaxis=dict(
        fixedrange=False  # allow y-axis zoom
    )
)

fig.show()


In [11]:
window = 20
n_std = 1.5

# Calculate the rolling mean and standard deviation of the merged['bin_minus_bybit']
merged["rolling_mean"] = merged['bin_minus_bybit'].rolling(window=30).mean()
merged["rolling_std"] = merged['bin_minus_bybit'].rolling(window=30).std()

# Calculate the z-score (number of standard deviations away from the rolling mean)
zscore = (merged['bin_minus_bybit'] - merged["rolling_mean"]) / merged["rolling_std"]

upper_band = merged["rolling_mean"] + n_std * merged["rolling_std"]
lower_band = merged["rolling_mean"] - n_std * merged["rolling_std"]

In [20]:
import plotly.graph_objects as go
import plotly.io as pio

# Force Plotly to open in the browser
pio.renderers.default = "browser"

# Parameters
window = 30
n_std = 1.5

# Calculate rolling stats
merged['rolling_mean'] = merged['bin_minus_bybit'].rolling(window=window).mean()
merged['rolling_std'] = merged['bin_minus_bybit'].rolling(window=window).std()

# Z-score
zscore = (merged['bin_minus_bybit'] - merged['rolling_mean']) / merged['rolling_std']

# Bands
upper_band = merged['rolling_mean'] + n_std * merged['rolling_std']
lower_band = merged['rolling_mean'] - n_std * merged['rolling_std']

# Start plot
fig = go.Figure()

# Funding spread
fig.add_trace(go.Scatter(
    x=merged.index,
    y=merged['bin_minus_bybit'],
    mode='lines',
    name='Funding Rate Spread',
    line=dict(color='red')
))

# Rolling mean
fig.add_trace(go.Scatter(
    x=merged.index,
    y=merged['rolling_mean'],
    mode='lines',
    name='Rolling Mean',
    line=dict(color='green', dash='dot')
))

# Upper and lower bands
fig.add_trace(go.Scatter(
    x=merged.index,
    y=upper_band,
    mode='lines',
    name='Upper Band',
    line=dict(color='blue', width=1),
    showlegend=True
))
fig.add_trace(go.Scatter(
    x=merged.index,
    y=lower_band,
    mode='lines',
    name='Lower Band',
    line=dict(color='blue', width=1),
    fill='tonexty',  # fills between this and previous trace
    fillcolor='rgba(173, 216, 230, 0.2)',
    showlegend=True
))

# Optional: Outlier dots where z-score > 1.5
# threshold = 1.5
# outliers = merged[np.abs(zscore) > threshold]
# fig.add_trace(go.Scatter(
#     x=outliers.index,
#     y=outliers['bin_minus_bybit'],
#     mode='markers',
#     name='Z > 1.5',
#     marker=dict(color='black', size=6, symbol='circle-open')
# ))

# Final layout
fig.update_layout(
    title='Funding Rate Spread: Binance - Bybit (With Rolling Mean & ±1.5 Std Bands)',
    xaxis_title='Date',
    yaxis_title='Annualized Funding Rate Difference',
    template='plotly_white',
    hovermode='x unified',
    dragmode='zoom',
    xaxis=dict(
        rangeslider=dict(visible=True),
        rangeselector=dict(
            buttons=[
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(step="all")
            ]
        )
    ),
    yaxis=dict(fixedrange=False)
)

fig.show()


In [29]:
import requests
requests.get("https://fapi.binance.com/fapi/v1/premiumIndex?symbol=BTCUSDT").json()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): fapi.binance.com:443
DEBUG:urllib3.connectionpool:https://fapi.binance.com:443 "GET /fapi/v1/premiumIndex?symbol=BTCUSDT HTTP/1.1" 200 None


{'symbol': 'BTCUSDT',
 'markPrice': '107761.60000000',
 'indexPrice': '107810.44891304',
 'estimatedSettlePrice': '107727.56675715',
 'lastFundingRate': '0.00000672',
 'interestRate': '0.00010000',
 'nextFundingTime': 1751299200000,
 'time': 1751286044000}

In [50]:
requests.get("https://api-testnet.bybit.com/v5/market/tickers?category=linear&symbol=BTCUSDT").json()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api-testnet.bybit.com:443
DEBUG:urllib3.connectionpool:https://api-testnet.bybit.com:443 "GET /v5/market/tickers?category=linear&symbol=BTCUSDT HTTP/1.1" 200 738


{'retCode': 0,
 'retMsg': 'OK',
 'result': {'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'lastPrice': '104418.70',
    'indexPrice': '107609.37',
    'markPrice': '104418.70',
    'prevPrice24h': '103711.00',
    'price24hPcnt': '0.006823',
    'highPrice24h': '104888.00',
    'lowPrice24h': '101400.00',
    'prevPrice1h': '104767.50',
    'openInterest': '837113.34',
    'openInterestValue': '87410286715.46',
    'turnover24h': '66114244.7851',
    'volume24h': '636.5620',
    'fundingRate': '-0.005',
    'nextFundingTime': '1751299200000',
    'predictedDeliveryPrice': '',
    'basisRate': '',
    'deliveryFeeRate': '',
    'deliveryTime': '0',
    'ask1Size': '5.446',
    'bid1Price': '104266.70',
    'ask1Price': '104418.70',
    'bid1Size': '0.002',
    'basis': '',
    'preOpenPrice': '',
    'preQty': '',
    'curPreListingPhase': ''}]},
 'retExtInfo': {},
 'time': 1751288868761}